<a href="https://colab.research.google.com/github/xCosmicx/ATA/blob/main/week14/token_classification_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Token Classification (Named Entity Recognition)

In this practical we will learn how to use the HuggingFace Transformers library to perform token classification.

Just like what we did in previous practical on BERT Transformer, we will use the DistilBERT transformer to classify each and every word (token) in a sentence.


## Install Transformers

Run the following cell to install the HuggingFace Transformers library.

In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.3 MB/s 
     |████████████████████████████████| 362 kB 70.7 MB/s 
     |████████████████████████████████| 6.6 MB 48.4 MB/s 
     |████████████████████████████████| 596 kB 75.4 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 1.1 MB 66.7 MB/s 
     |████████████████████████████████| 212 kB 71.7 MB/s 
     |████████████████████████████████| 140 kB 64.8 MB/s 
     |████████████████████████████████| 127 kB 76.5 MB/s 
     |████████████████████████████████| 144 kB 71.7 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 271 kB 76.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing

## Get the data

In this lab, we will use the CoNLL-2003 dataset. 

In [ ]:
!wget https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/datasets/conll2003.zip
!unzip conll2003.zip

--2022-07-19 02:16:19--  https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/datasets/conll2003.zip
Resolving nyp-aicourse.s3.ap-southeast-1.amazonaws.com (nyp-aicourse.s3.ap-southeast-1.amazonaws.com)... 3.5.146.143
Connecting to nyp-aicourse.s3.ap-southeast-1.amazonaws.com (nyp-aicourse.s3.ap-southeast-1.amazonaws.com)|3.5.146.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 795342 (777K) [application/zip]
Saving to: ‘conll2003.zip’

conll2003.zip       100%[===================>] 776.70K   284KB/s    in 2.7s    

2022-07-19 02:16:22 (284 KB/s) - ‘conll2003.zip’ saved [795342/795342]

Archive:  conll2003.zip
  inflating: token_test.txt          
  inflating: token_train.txt         


## Process the data 

The data file is in CoNLL format: 

```
sentence1-word1  PPTag-1-1  ChunkTag-1-1  NERTag-1-1
sentence1-word2  PPTag-1-2  ChunkTag-1-2  NERTag-1-2
sentence1-word3  PPTag-1-3  ChunkTag-1-3  NERTag-1-3
<empty line>
sentence2-word1  PPTag-2-1  ChunkTag-2-1  NERTag-2-1
sentence2-word2  PPTag-2-2  ChunkTag-2-2  NERTag-2-2
...
sentence2-wordn  PPTag-2-n  ChunkTag-2-n  NERTag-2-n
<empty line>
...
```

For example, the sentence "U.N. official Ekeus heads for Baghdad." will be represented as follow in CoNLL format: 

```
U.N.      NNP  I-NP  I-ORG
official  NN   I-NP  O
Ekeus     NNP  I-NP  I-PER
heads     VBZ  I-VP  O
for       IN   I-PP  O
Baghdad   NNP  I-NP  I-LOC
.         .    O     O
```

We define a function to read the data file line by line and combined lines that belong to a sentence into a list of words and list of tags. 

As we are only interested in the Named Entity Recognition (NER) tags, we will only extract tags from column_index 3.

In [ ]:
# This function returns a 2D list of words and a 2D list of labels
# corresponding to each word.

def load_conll(filepath, delimiter=' ', word_column_index=0, label_column_index=3):
    all_texts = []
    all_tags = []

    texts = []
    tags = []

    # Opens the file.
    #
    with open(filepath, "r") as f:

        # Loops through each line 
        for line in f:

            # Split each line by its delimiter (default is a space)
            tokens = line.split(delimiter)

            # If the line is empty, treat it as the end of the
            # previous sentence, and construct a new sentence
            #
            if len(tokens) == 1:
                # Append the sentence
                # 
                all_texts.append(texts)
                all_tags.append(tags)

                # Create a new sentence
                #
                texts = []
                tags = []
            else:
                # Not yet end of the sentence, continue to add
                # words into the current sentence
                #
                thistext = tokens[word_column_index].replace('\n', '')
                thistag = tokens[label_column_index].replace('\n', '')

                texts.append(thistext)
                tags.append(thistag)

    # Insert the last sentence if it contains at least 1 word.
    #
    if len(texts) > 0:
        all_texts.append(texts)
        all_tags.append(tags)

    # Return the result to the caller
    #
    return all_texts, all_tags


We will now process our files with the function and examine the outputs.

In [ ]:
train_texts, train_tags = load_conll("token_train.txt")
val_texts, val_tags = load_conll("token_test.txt")

In [ ]:
print(train_texts[:3])
print(train_tags[:3])

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn'], ['BRUSSELS', '1996-08-22']]
[['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'], ['B-PER', 'I-PER'], ['B-LOC', 'O']]


## Tokenization

Now we have our texts and labels. Before we can feed the texts and labels into our model for training, we need to tokenize our texts and also encode our labels into numeric forms.

We first define the token labels we need and define the mapping to a numeric index.


In [ ]:
# Define a list of unique token labels that we will recognize
#
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

# Define a dictionary to map txt label to numeric label
label2id = {label:i for i, label in enumerate(label_names)}

We will also import the tokenizer.

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = 'distilbert-base-uncased'
# Initialize the DistilBERT tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, is_fast=True)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Before we tokenize our texts,  let's look at a potential problem that can happen we do tokenization. Transformers model like Bert or DistilBert uses WordPiece tokenization, meaning that a single word can sometimes be split into multiple tokens (this is done to solve the out-of-vocabulary problem for rare words). For example, DistilBert’s tokenizer would split the `["Nadim", "Ladki"]` into the tokens `[[CLS], "na", "##im","lad", ##ki", [SEP]]`. This is a problem for us because we have exactly one tag per token in the original dataset. If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels, as illustrated below:

Before tokenization with WordPiece, it is one to one matching between tokens and tags:

```
tokens = ["Nadim", "Ladki"]
labels = ['B-PER', 'I-PER']
```

After tokenization with WordPiece, there is no more one-to-one match between them: 
```
tokens = ["[CLS]", "nad", "##im", "lad", "##ki", "[SEP]"]
labels = ['B-PER', 'I-PER']
```

One way to handle this is to only train on the tag labels for the first subtoken of a split token. We can do this in Transformers by setting the labels we wish to ignore to -100. We will also ignore special tokens like `[CLS]` and `[SEP]`. In the example above, if the label for 'Nadim' is 1 (index for B-PER) and 'Ladki' is 2 (index for I-PER), we would set the labels as follows: 

```
tokens = ["[CLS]", "nad", "##im", "lad", "##ki", "[SEP]"]
labels = [-100, 1, -100, 2, -100, -100]
```

But how do we know which are sub-tokens and the special tokens to ignore? Fortunately, the Huggingface tokenize provides us a way to do it: `word_ids`. `word_ids` will tell us which word each token comes from, as well as which words are special tokens (e.g. `[CLS]`). 

For example, the `word_ids` for the following tokens will be: 

```
tokens = ["[CLS]", "nad", "##im", "lad", "##ki", "[SEP]"]
word_ids = [None, 0, 0, 1, 1, None]
```

`None` means it is a special token. You can see that `"nad"`, `"##im"` are both having word_ids `0`, which means both tokens comes from the word at index 0, i.e. `"nadim"`. Similarly, `"lad"` and `"##ki"` have word_ids of `1`, which means both comes from the 2nd word, i.e. word at index 1.


So we can simply use the following logic to decide how to label each of the processed tokens (i.e tokens that have already processed by the tokenizer, and consist of special tokens and subtokens):
- if a token has a `word_id` of `None`, we will set its corresponding label to `-100`. - if the `word_id` of the token appears the 1st time, i.e. different from previous `word_id`, set the label of the token to the corresponding original label. 
- if the `word_id` is the same as previous `word_id`, set the label for the tokens to `-100`

The following function `tokenize_and_align_labels()` takes in original tags and encode it according to the logic described above. 

Note that for this version of HuggingFace, we need to supply the label as part of the dictionary. That is why we create additional entry `tokenized_inputs['labels']` to hold the labels. 

In [ ]:
# set the max sequence length and require padding
max_length=128
padding=True

def tokenize_and_align_labels(texts, all_tags):
    
    tokenized_inputs = tokenizer(
        texts,
        max_length=max_length,
        padding=padding,
        truncation=True,
        is_split_into_words=True,
    )

    labels = []

    for i, tags in enumerate(all_tags):
        word_ids = tokenized_inputs[i].word_ids
        tokens = tokenized_inputs[i].ids
        previous_word_idx = None
        label_ids = []
       
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(int(label2id[tags[word_idx]]))
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
                
            previous_word_idx = word_idx

        labels.append(label_ids)
        
        tokenized_inputs['labels'] = labels
        
    return tokenized_inputs, labels

In [ ]:
train_encodings, train_labels = tokenize_and_align_labels(train_texts, train_tags)

In [ ]:
val_encodings, val_labels = tokenize_and_align_labels(val_texts, val_tags)

In [ ]:
import tensorflow as tf

batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(batch_size)

Run the following cell below to see the first few samples of the train dataset to see if they looks all right.

In [ ]:
iterator = iter(train_dataset)

for i in range(3):
    print (train_texts[i])
    print(iterator.get_next())
    print ("---")

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
({'input_ids': <tf.Tensor: shape=(16, 128), dtype=int32, numpy=
array([[  101,  7327, 19164, ...,     0,     0,     0],
       [  101,  2848, 13934, ...,     0,     0,     0],
       [  101,  9371,  2727, ...,     0,     0,     0],
       ...,
       [  101,  1996,  7327, ...,     0,     0,     0],
       [  101,  8351,  2031, ...,     0,     0,     0],
       [  101,  2329,  6617, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(16, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>, 'labels': <tf.Tensor: shape=(16, 128), dtype=int32, numpy=
array([[-100,    3,    0, ..., -100, -100, -100],
       [-100,    1,    2, ..., -100, -100, -100],
       [-100,    5,    0, ..., -100, -100, -100],

## Train our Token Classification Model

We will now load the pretrained model and configure the required token labels for the model.


In [ ]:
from transformers import TFAutoModelForTokenClassification, AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint, num_labels=len(label_names))

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint, 
    config=config
)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForTokenClassification: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

Let’s double-check that our model has the right number of labels:

In [ ]:
model.config.num_labels

9

In [ ]:
from transformers import create_optimizer

num_epochs = 1
num_train_steps = len(train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Huggingface model can actually compute loss internally — if you compile without a loss and supply your labels in the input dictionary (as we do in our datasets), then the model will train using that internal loss, which will be appropriate for the task and model type you have chosen.

In [ ]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=num_epochs
)

937/937 [==============================] - 231s 235ms/step - loss: 0.2044 - val_loss: 0.0988


## Evaluation

The traditional framework used to evaluate token classification prediction is `seqeval`. To use this metric, we first need to install the `seqeval` library:

In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=1aa5b215cb6b46fb3ae6aa5f2eb538aa51921dbb57894e8ef76a25fff760f194
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
from datasets import load_metric

metric = load_metric('seqeval')

In the following codes, we use our model to predict our val_dataset, in batches. For each batch of tf dataset, we have two parts: 1st contains the `input_ids`, `attention_masks`, and `labels`, while the second one is target label. We will only use the 1st part for prediction, i.e. `batch[0]`

Also while looping through the list of predicted label for each token, we will ignore those positions that is labeled "-100". 

In [ ]:
import numpy as np

all_predictions = []
all_labels = []

for batch in val_dataset:
    logits = model.predict(batch[0])["logits"]
    labels = batch[0]["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(label_names[predicted_idx])
            all_labels.append(label_names[label_idx])

metric.compute(predictions=[all_predictions], references=[all_labels])

{'LOC': {'f1': 0.884478672985782,
  'number': 1667,
  'precision': 0.8736102984201287,
  'recall': 0.8956208758248351},
 'MISC': {'f1': 0.6895043731778425,
  'number': 702,
  'precision': 0.7059701492537314,
  'recall': 0.6737891737891738},
 'ORG': {'f1': 0.8002326257633031,
  'number': 1661,
  'precision': 0.7739032620922385,
  'recall': 0.8284166164960867},
 'PER': {'f1': 0.9515282494597098,
  'number': 1616,
  'precision': 0.9494762784966112,
  'recall': 0.9535891089108911},
 'overall_accuracy': 0.9720072875361697,
 'overall_f1': 0.8547173113950639,
 'overall_precision': 0.8448096885813149,
 'overall_recall': 0.8648600779312788}

In [ ]:
model.save_pretrained('token_model')

## Test on your sentence


In [ ]:
def infer_tokens(text, model, tokenizer):
    # here we assume the text has not been splitted into individual words
    text = text.split()
    
    encodings = tokenizer(
        [text],
        padding=True,
        truncation=True,
        is_split_into_words=True,
        return_tensors='tf')
    
    logits = model(encodings)[0] # assume only a single prediction
    preds = np.argmax(logits, axis=-1)[0]

    # as the prediction is on individual tokens, including subtokens, 
    # we need to group subtokens belonging to the same word together
    # again, we use the word_ids to help us here
    previous_word_idx = None
    word_ids = encodings[0].word_ids
    labels = []
    for i, word_idx in enumerate(word_ids):
        # we check if the word_id different from previous one, then it is a new word
        # we also need to check if the word_id is not None so that we won't include it
        if word_idx != previous_word_idx and word_idx != None:
            labels.append(label_names[preds[i]])
        # update the previous_word_idx to current word_id
        previous_word_idx = word_idx

    return text, labels

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = TFAutoModelForTokenClassification.from_pretrained('token_model')

Some layers from the model checkpoint at token_model were not used when initializing TFDistilBertForTokenClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at token_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sample_text = 'Ashish Vaswani has developed the transformer architecture during his time at Google.'
infer_tokens(sample_text, model, tokenizer)

(['Ashish',
  'Vaswani',
  'has',
  'developed',
  'the',
  'transformer',
  'architecture',
  'during',
  'his',
  'time',
  'at',
  'Google.'],
 ['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG'])

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !cp token_model.zip drive/MyDrive